In [56]:
import pandas as pd
import os

# Specify the Excel output filename
excel_filename="2025-04-10 Political_Cartoons.xlsx"

# Specify the two folders you want to search
folders_to_search = ['political cartoons saved\\']

#specift image endings: include jpg, jpeg, and png
file_endings='*.*g'

#Specify TEMP image folder (resized or renamed)
temp_folder='TEMP\\'

In [23]:
from pathlib import Path

def find_pdf_files(directories):
    pdf_files = [
        str(pdf_file) for directory in directories 
        for pdf_file in Path(directory).rglob(file_endings)
    ]
    
    return pdf_files

pdf_file_paths = find_pdf_files(folders_to_search)

In [27]:
# Create a dataframe with newspaper & filename and ending.

df=pd.DataFrame([
    {'path':i,"newspaper":i.rsplit("\\")[-2],
     "name":i.rsplit("\\")[-1].split(".")[0],
     "ending":os.path.splitext(i)} for i in pdf_file_paths
   ])

filename)

df.sort_values("path",inplace=True)
df.reset_index(drop=True,inplace=True)
df.to_excel(excel_filename)


df.head()

,path,newspaper,name,ending
0,political cartoons saved\Corriere della Sera\C...,Corriere della Sera,Corriere_1,jpg
1,political cartoons saved\Corriere della Sera\C...,Corriere della Sera,Corriere_10,jpg
2,political cartoons saved\Corriere della Sera\C...,Corriere della Sera,Corriere_11,jpg
3,political cartoons saved\Corriere della Sera\C...,Corriere della Sera,Corriere_12,jpg
4,political cartoons saved\Corriere della Sera\C...,Corriere della Sera,Corriere_13,jpg


In [51]:
#Rename Images

repl_dict={"Corriere della Sera":"Corriere",
          "The Guardian":"Guardian"}

df["new_name"]=df.newspaper.replace(repl_dict)+ '_' + (df.groupby('newspaper').cumcount() + 1).astype(str)

# Assuming df is your DataFrame and temp_folder is defined
for index, row in df.iterrows():
    old_folder = os.path.dirname(row["path"]) + "\\"
    old_file_path = os.path.join(old_folder, f"{row['name']}.{row['ending']}")
    new_file_path = os.path.join(temp_folder, f"{row['new_name']}.{row['ending']}")

    if row["name"].split("_")[0] in repl_dict.values():
        # Create temp_folder if it doesn't exist
        os.makedirs(temp_folder, exist_ok=True)

        # Copy file if it doesn't already exist
        if not os.path.exists(new_file_path):
            with open(old_file_path, 'rb') as src_file, open(new_file_path, 'wb') as dst_file:
                dst_file.write(src_file.read())
            df.loc[index, 'new_path'] = new_file_path
            print(new_file_path)
        else:
            print('File already exists:', new_file_path)

TEMP\Corriere_1.jpg
TEMP\Corriere_2.jpg
TEMP\Corriere_3.jpg
TEMP\Corriere_4.jpg
TEMP\Corriere_5.jpg
TEMP\Corriere_6.jpg
TEMP\Corriere_7.jpg
TEMP\Corriere_8.jpg
TEMP\Corriere_9.jpg
TEMP\Corriere_10.jpg
TEMP\Corriere_11.jpg
TEMP\Corriere_12.jpg
TEMP\Corriere_13.jpg
TEMP\Corriere_14.jpg
TEMP\Corriere_15.jpg
TEMP\Guardian_1.jpg
TEMP\Guardian_2.jpg
TEMP\Guardian_3.jpg
TEMP\Guardian_4.jpg
TEMP\Guardian_5.jpg
TEMP\Guardian_6.jpg
TEMP\Guardian_7.jpg
TEMP\Guardian_8.jpg
TEMP\Guardian_9.jpg
TEMP\Guardian_10.jpg
TEMP\Guardian_11.jpg
TEMP\Guardian_12.jpg
TEMP\Guardian_13.jpg
TEMP\Guardian_14.jpg
TEMP\Guardian_15.jpg
TEMP\Guardian_16.jpg
TEMP\Guardian_17.jpg
TEMP\Guardian_18.jpg
TEMP\Guardian_19.jpg
TEMP\Guardian_20.jpg
TEMP\Guardian_21.jpg
TEMP\Guardian_22.jpg
TEMP\Guardian_23.jpg
TEMP\Guardian_24.jpg
TEMP\Guardian_25.jpg
TEMP\Guardian_26.jpg
TEMP\Guardian_27.jpg
TEMP\Guardian_28.jpg
TEMP\Guardian_29.jpg
TEMP\Guardian_30.jpg
TEMP\Guardian_31.jpg
TEMP\Guardian_32.jpg
TEMP\Guardian_33.jpg


In [59]:
# resize function for images:


from PIL import Image as PILImage

def resize_image(img_path, max_height):
    try:
        """Resize the image to the specified maximum height while maintaining aspect ratio and return the new filename."""
        with PILImage.open(img_path) as img:
            aspect_ratio = img.width / img.height
            new_height = max_height
            new_width = int(new_height * aspect_ratio)
            
            # Create a new filename with a suffix
            base, ext = os.path.splitext(img_path)
            new_filename = f"{base}_resized{ext}"
            
            # Save the resized image
            img = img.resize((new_width, new_height))
            img.save(new_filename)
            
        return new_filename, new_width, new_height
    except:
        return img_path, max_height, max_height


In [61]:
## Put files into excel sheet

import openpyxl
from openpyxl.drawing.image import Image
import os


df[["newspaper","name","new_name"]].to_excel(excel_filename)

# Load or create an Excel workbook
workbook = openpyxl.load_workbook(excel_filename) 

#workbook = openpyxl.Workbook()
sheet = workbook.active

# Loop through the image filenames and insert them into the Excel sheet

for index, img_path in enumerate(df["new_path"]):
    if isinstance(img_path,str):
        # Check if the image file exists
        if os.path.exists(img_path):
            
            new_filename, new_width, new_height = resize_image(img_path, max_height=150)            # Create an Image object
            
            img = Image(new_filename)
            # After inserting the image
            img = Image(new_filename)
            img.width = new_width
            img.height = new_height
            
            # Define the cell where the image will be placed (e.g., B1, B2, B3, ...)
            cell_location = f'M{index + 2}'
            
            # Add the image to the specified cell
            sheet.add_image(img, cell_location)

            # Set the row height to match the image height
            sheet.row_dimensions[index + 2].height = new_height
        else:
            print(f"Image not found: {img_path}")

# Save the workbook
workbook.save(excel_filename)
